# CS 237: Homework 4 Programming Exercises

In [9]:
# add your imports here

# here are some examples of imports
import matplotlib.pyplot as plt   # for plotting
import numpy as np                # for simulating random choices
from collections import Counter   # for aggregating the results

In the following exercises, we will run simulations of a recommender system that recommends movies to users, as we discussed in class on 9/23, and in this Piazza post: https://piazza.com/class/ksg5aj427qney?cid=126

## Step 1: Baseline Movie Ratings

We will start by generating baseline star-ratings to movies.  We assume there are $k$ movies in the system, and their ratings are distributed uniformly between 1.0 stars (terrible) and 5.0 stars (awesome).

In [2]:
k = 10000
movies = np.random.uniform(1.0, 5.0, k)
#print (movies)

## Exercise 1: Simulating User Populations

In this exercise, we ask you to now simulate ratings from two user different populations.  

User population 1 consists of random movie-watchers.  They pick movies completely at random, without regard to the underlying rating. After watching movie $i$ with rating $r_i$, they then generate a user rating

$u = r_i+ \delta$

where $\delta$ is uniform on [-1.0, 1.0]

and constrain that rating to be in the valid range [1.0, 5.0]

$$u = \max (1.0, u) \textit{   # round up to 1.0 if needed} $$
$$u = \min (5.0, u) \textit{   # round down to 5.0 if needed} $$

A rating is recorded as the tuple $(i, u)$.

Produce a list of 50,000 ratings produced by user population 1 and report the average rating the users generated as average1. 

====================

User population 2 consists of more discriminating users.  They choose movies proportionally to the underlying rating using the following selection probabilities:

$$\Pr[\text{Select movie } i]  = \frac{r_i} {\sum_j {r_j}}$$ 

They then generate a user rating using the same method as user population 1, except their $\delta$ is uniform on [-0.5, 0.5]. 

Produce a list of 50,000 ratings produced by user population 2 and report the average rating the users generated as average2. 

[Hint:  Implementation time-saver:  consider using the numpy function random.choice() to implement selections for User population 2]


In [3]:
#  Your code here to compute average1 and average2

####################population1####################

u_total_1 = 0

for i in range (50000):
    u_1 = np.random.choice(movies, 1) + np.random.uniform(-1.0, 1.0, 1)
    if u_1 <= 1:
        u_1 = max(1.0, u_1)
    if u_1 >= 5:
        u_1 = min(5.0, u_1)
    u_total_1 += u_1

average1 = u_total_1/50000

####################population2####################

u_total_2 = 0

rj = 0
pr_movies = []

rj = sum(movies)

for i in range (10000):
    pr_movies.append(movies[i]/rj)

for i in range (50000):
    u_2 = np.random.choice(movies, 1, p = pr_movies) + np.random.uniform(-0.5, 0.5, 1)
    if u_2 <= 1:
        u_2 = max(1.0, u_2)
    if u_2 >= 5:
        u_2 = min(5.0, u_2)
    u_total_2 += u_2

average2 = u_total_2/50000

In [12]:
#  Please execute this next line by un-commenting it:
print(average1, average2)

[2.98353801] [3.41739585]


## Exercise 2: Basic Predictions

In this exercise, we ask you to now assess prediction error from different models.  When a user watches a movie $i$ (with rating $r_i$), each model will make a prediction $p$ and the user will provide a rating $u$.

Model A only knows about users:  it makes a conservative prediction of $p = 3.0$ for any movie watched by user population 1 and $p = 4.0$ for any movie watched by user population 2.  

Model B only knows about movies:  it predicts a value of $p = r_i$ whenever movie $i$ is watched (by any user). 

Simulate the two models on 50,000 trials of the following form.

  - choose a user type uniformly at random between Population 1 and Population 2 (50/50)
  - choose a movie at random following that user population's selection method
  - compute a predicted rating $p_A$ made by model A
  - compute a predicted rating $p_B$ made by model B
  - compute a rating $u$ the user generates for that movie
  
Store the outcomes of all the trials in a list, where the outcome of a trial as a 4-tuple:  (movieID, $u, p_A, p_B$).
  

Finally, assess the models on how much error it made over all predictions.  For this assignment we'll consider the the mean squared error (MSE).
The model error for one trial (for model A) is defined to be $\epsilon = p_A - u$ and the squared error for one trial is $\epsilon^2$.  The mean squared error over all trials (for model A) is the average of all 50,000 squared errors $\epsilon^2$. 

Compute and report the MSE for Model A and similarly, for Model B.




In [8]:
#  Your code here to run the 50,000 trials and compute MSE_A and MSE_B

total_model_error_square_a = 0
total_model_error_square_b = 0
movieID = 0
u = 0
p_a = 0
p_b = 0

for i in range(50000):
    
    if np.random.randint(1, 3) == 1:
        
        ###p_a###
        p_a = 3.0
        
        ###p_b###
        p_b = np.random.choice(movies, 1)
        
        ###movieID#############        
        for i in range(10000):
            if movies[i] == p_b:
                movieID = i
        
        ###u################################
        u = p_b + np.random.uniform(-1.0, 1.0)
        if u <= 1:
            u = max(1.0, u)
        if u >= 5:
            u = min(5.0, u)

    ########################        
            
    else:
        
        ###p_a###
        p_a = 4.0
        
        ###p_b###
        rj = 0
        pr_movies = []
        
        for i in range (10000):
            rj += movies[i]
        for i in range (10000):
            pr_movies.append(movies[i]/rj)
            
        p_b = np.random.choice(movies, 1, p = pr_movies)
        
        ###movieID#############
        for i in range(10000):
            if movies[i] == p_b:
                movieID = i
        
        ###u################################
        u = p_b + np.random.uniform(-0.5, 0.5)
        if u <= 1:
            u = max(1.0, u)
        if u >= 5:
            u = min(5.0, u)
            
    outcomes = (movieID, u, p_a, p_b)
    model_error_a = p_a - u
    model_error_b = p_b - u
    model_error_square_a = model_error_a * model_error_a
    model_error_square_b = model_error_b * model_error_b
    total_model_error_square_a += model_error_square_a
    total_model_error_square_b += model_error_square_b
    
MSE_A = total_model_error_square_a/50000
MSE_B = total_model_error_square_b/50000

In [9]:
#  Please execute this next line by un-commenting it:
print(MSE_A, MSE_B)

[1.50871966] [0.21056244]


## Exercise 3: Can you do better?

Based on your experiments in parts 1 and 2, can you make simple changes to improve upon the existing models?

Improve model A just by changing the hard-wired prediction values $p_1 = 3.0$ and/or $p_2 = 4.0$.  Print the new settings for $p_1$ and $p_2$ and print the MSE for this new Model C.

It's harder to improve upon model B.  Tell us why in a couple of sentences.
Alternatively, impress us by showing that it is possible to do a little better than just predicting $r_i$.




In [8]:
#  Your code here to compute MSE_C
#  Your code here to run the 50,000 trials and compute MSE_A and MSE_B

total_model_error_square_a = 0
total_model_error_square_b = 0
movieID = 0
u = 0
p_a = 0
p_b = 0

for i in range(50000):
    
    if np.random.randint(1, 3) == 1:
        
        ###p_a###
        p_a = average1
        
        ###p_b###
        p_b = np.random.choice(movies, 1)
        
        ###movieID#############        
        for i in range(10000):
            if movies[i] == p_b:
                movieID = i
        
        ###u################################
        u = p_b + np.random.uniform(-1.0, 1.0)
        if u <= 1:
            u = max(1.0, u)
        if u >= 5:
            u = min(5.0, u)

    ########################        
            
    else:
        
        ###p_a###
        p_a = average2
        
        ###p_b###
        rj = 0
        pr_movies = []
        
        for i in range (10000):
            rj += movies[i]
        for i in range (10000):
            pr_movies.append(movies[i]/rj)
            
        p_b = np.random.choice(movies, 1, p = pr_movies)
        
        ###movieID#############
        for i in range(10000):
            if movies[i] == p_b:
                movieID = i
        
        ###u################################
        u = p_b + np.random.uniform(-0.5, 0.5)
        if u <= 1:
            u = max(1.0, u)
        if u >= 5:
            u = min(5.0, u)
            
    model_error_a = p_a - u
    model_error_square_a = model_error_a * model_error_a
    total_model_error_square_a += model_error_square_a
    
MSE_C = total_model_error_square_a/50000

#  Please execute this next line by un-commenting it:
print(p_1, p_2, MSE_C)

3.0 3.4 [1.29403057]


In [ ]:
#  Your 1-2 answer below for why it's hard to improve much on Model B, or, code for how you did it.
# Because for model B, we don't have a way to find the average rating like we did in exercise for population one
# and two. And the rating is totaly random, so it's very hard to predict.